In [1]:
%pip install langchain-core langchain-google-genai langchain-text-splitters langchain-chroma pypdf python-dotenv

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
from typing import List, Dict, Any
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda, RunnableMap
from langchain_core.messages import HumanMessage, AIMessage # HumanMessage import 추가
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_chroma import Chroma
from dotenv import load_dotenv

# --- 환경 변수 로드 ---
load_dotenv()
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
if not GOOGLE_API_KEY:
    raise ValueError("환경 변수 'GOOGLE_API_KEY'를 설정해주세요.")

chat = ChatGoogleGenerativeAI(model="gemini-2.5-flash", api_key=GOOGLE_API_KEY, temperature=0.0)
embedding_model = GoogleGenerativeAIEmbeddings(model="text-embedding-004", api_key=GOOGLE_API_KEY)

In [3]:
# --- 2. ChromaDB 생성 (클리닝 단계 제거) ---
pdf_path = r"C:\Users\ziclp\OneDrive\Desktop\LLM Assignment\1week\PRACTICE1\chap9\2040_seoul_plan.pdf" 
persist_directory = './chroma_store_seoul_raw' # 클리닝되지 않은 데이터를 위한 새 폴더
os.makedirs(persist_directory, exist_ok=True) 

# 1. PDF 로드 
loader = PyPDFLoader(pdf_path)
raw_docs = loader.load() # ⬅️ 노이즈가 포함된 원본 문서 리스트

# 2. 텍스트 분할 (원본 문서를 그대로 분할)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, 
    chunk_overlap=400, # ⬅️ 오버랩 400 유지
    separators=["\n\n\n", "\n\n", "\n", " ", ""] 
)
all_splits = text_splitter.split_documents(raw_docs) # ⬅️ 클리닝 없이 raw_docs를 분할
print(f"✅ 노이즈 포함 유효 청크 개수: {len(all_splits)}개")

# 3. ChromaDB 재생성 및 색인
vectorstore = Chroma.from_documents(
    documents=all_splits,
    embedding=embedding_model,
    persist_directory=persist_directory
)
print(f"✅ ChromaDB 재생성 및 색인 완료: {persist_directory} (노이즈 포함)")

✅ 노이즈 포함 유효 청크 개수: 312개
✅ ChromaDB 재생성 및 색인 완료: ./chroma_store_seoul_raw (노이즈 포함)


In [4]:
# --- 3. LCEL 체인 및 Retriever 정의 ---

# A. Retriever 정의 (MMR 유사성 비중 극대화)
retriever = vectorstore.as_retriever(
    search_type="mmr", 
    search_kwargs={'k': 10, 'fetch_k': 50, 'lambda_mult': 0.9} 
)

# B. Document Chain 정의
def stuff(docs: List[Document]) -> str:
    return "\n\n".join(d.page_content for d in (docs or []))

prompt = ChatPromptTemplate.from_messages([
    ("system",
     "아래 문맥(Context)에 근거해 답하세요. 문맥에 없으면 "
     "'제공된 문서에는 해당 정보가 없습니다.'라고 답하세요.\n\n{context}"),
    MessagesPlaceholder("messages"),
])

document_chain = (
    RunnableMap({
        "context": RunnableLambda(lambda x: stuff(x.get("context", []))),
        "messages": RunnableLambda(lambda x: x.get("messages", [])),
    })
    | prompt
    | chat
)

# --- 4. 최종 검색 및 답변 생성 ---
user_query = "서울시의 온실가스저감정책 및 전략을 자세히 설명해줘"

print("\n--- 🔍 최종 검색 결과 (온실가스) ---")
# 1. 검색 실행 
docs = retriever.invoke(user_query)

# 2. 검색된 문서 내용 출력 
for d in docs:
    page_info = d.metadata.get('page_label') or d.metadata.get('page')
    print(d.page_content[:200] + '...')
    print(f"[{page_info}]")
    print('--------------------')

# 3. LLM 호출 및 답변 생성
chat_history = [HumanMessage(content=user_query)]
answer = document_chain.invoke(
    {
        "messages": chat_history,
        "context": docs, 
    }
)

# 4. 출력
print("\n--- 🤖 최종 답변 ---")
print(answer.content)


--- 🔍 최종 검색 결과 (온실가스) ---
부록계획단계별 참여진...
[195]
--------------------
[표 3-3] 주택·정비 부문별 전략계획...
[75]
--------------------
[표 3-4] 경제·산업 부문별 전략계획...
[81]
--------------------
[표 3-5] 기후·환경 부문별 전략계획...
[86]
--------------------
[표 3-6] 안전·방재 부문별 전략계획...
[91]
--------------------
제3절 수립과정 및 추진체계11...
[19]
--------------------
제3절 광역적 협력 거버넌스 강화183...
[191]
--------------------
190[부록] 계획단계별 참여진...
[198]
--------------------
94제3장 부문별 전략계획...
[102]
--------------------
제5장생활권계획
제1절 생활권계획의 개요제2절 생활권 구분 및 설정제3절 권역별 구상...
[145]
--------------------

--- 🤖 최종 답변 ---
제공된 문서에는 해당 정보가 없습니다.
